In [ ]:
import cupy
import dask.array as da
import numpy as np
import math

from numba import guvectorize
from sgkit.typing import ArrayLike
from dask.diagnostics import ProgressBar, ResourceProfiler

def pairwise_distance(
    x: ArrayLike,
    metric: str = "euclidean",
) -> np.ndarray:
    x = da.asarray(x)
    x_distance = da.blockwise(
        # Lambda wraps reshape for broadcast
        euclidean,
        "jk",
        x,
        "ji",
        x,
        "ki",
        dtype="float64",
        concatenate=True,
    )
    x_distance = da.triu(x_distance, 1) + da.triu(x_distance).T
    return x_distance.compute()


@guvectorize(  # type: ignore
    [
        "void(float32[:, :], float32[:, :], float32[:, :])",
        "void(float64[:, :], float64[:, :], float64[:, :])",
        "void(int8[:, :], int8[:, :], float64[:, :])",
    ],
    "(m, k),(n, k)->(m, n)", target='cuda'
)
def euclidean(x: ArrayLike, y: ArrayLike, out: ArrayLike) -> None:
    o = x.shape[-1]
    m = x.shape[0]
    n = y.shape[0]

    for j in range(m):
        for k in range(n):
            square_sum = 0.0
            for i in range(o):
                if x[j, i] >= 0 and y[k, i] >= 0:
                    square_sum += (x[j, i] - y[k, i]) ** 2
            out[j, k] = math.sqrt(square_sum)

In [ ]:
# rs = da.random.RandomState(0)
rs = da.random.RandomState(RandomState=cupy.random.RandomState)
x = rs.normal(10, 1, size=(500, 500), chunks=(100, -1))
x

In [ ]:
%%time
with ProgressBar(), ResourceProfiler() as prof:
    pairwise_distance(x)
prof.visualize()

In [ ]:
x = y = np.arange(9).reshape(3, 3)

In [ ]:
x*y

In [ ]:
x

In [ ]:
from numba import cuda, float32

# Controls threads per block and shared memory usage.
# The computation will be done on blocks of TPBxTPB elements.
TPB = 16

@cuda.jit
def fast_matmul(A, B, C):
    # Define an array in the shared memory
    # The size and type of the arrays must be known at compile time
    sA = cuda.shared.array(shape=(TPB, TPB), dtype=float32)
    sB = cuda.shared.array(shape=(TPB, TPB), dtype=float32)

    x, y = cuda.grid(2)

    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    bpg = cuda.gridDim.x    # blocks per grid

    if x >= C.shape[0] and y >= C.shape[1]:
        # Quit if (x, y) is outside of valid C boundary
        return

    # Each thread computes one element in the result matrix.
    # The dot product is chunked into dot products of TPB-long vectors.
    tmp = 0.
    for i in range(bpg):
        # Preload data into shared memory
        sA[tx, ty] = A[x, ty + i * TPB]
        sB[tx, ty] = B[tx + i * TPB, y]

        # Wait until all threads finish preloading
        cuda.syncthreads()

        # Computes partial product on the shared memory
        for j in range(TPB):
            tmp += sA[tx, j] * sB[j, ty]

        # Wait until all threads finish computing
        cuda.syncthreads()

    C[x, y] = tmp

In [ ]:
import math

import dask.array as da
import numpy as np


def pairwise_distance(x):
    x = da.asarray(x)
    x_distance = da.blockwise(
        lambda _x, _y: euclidean(_x[:, None, :], _y),
        "jk",
        x,
        "ji",
        x,
        "ki",
        dtype="float64",
        concatenate=True,
    )
    x_distance = da.triu(x_distance, 1) + da.triu(x_distance).T
    return x_distance.compute()

@guvectorize(  # type: ignore
    [
        "void(float32[:], float32[:], float32[:])",
        "void(float64[:], float64[:], float64[:])",
        "void(int8[:], int8[:], float64[:])",
    ],
    "(n),(n)->()", target='cuda'
)
def euclidean(x, y, out) -> None:
    square_sum = 0.0
    m = x.shape[0]

    for i in range(m):
        if x[i] >= 0 and y[i] >= 0:
            square_sum += (x[i] - y[i]) ** 2
    out[0] = math.sqrt(square_sum)

rs = da.random.RandomState(RandomState=cupy.random.RandomState)
x = rs.normal(10, 1, size=(500, 500), chunks=(100, -1))
pairwise_distance(x)

In [1]:
from numba import cuda, float32

# Controls threads per block and shared memory usage.
# The computation will be done on blocks of TPBxTPB elements.
TPB = 16

@cuda.jit
def fast_matmul(A, B, C):
    # Define an array in the shared memory
    # The size and type of the arrays must be known at compile time
    sA = cuda.shared.array(shape=(TPB, TPB), dtype=float32)
    sB = cuda.shared.array(shape=(TPB, TPB), dtype=float32)

    x, y = cuda.grid(2)

    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    bpg = cuda.gridDim.x    # blocks per grid

    if x >= C.shape[0] and y >= C.shape[1]:
        # Quit if (x, y) is outside of valid C boundary
        return

    # Each thread computes one element in the result matrix.
    # The dot product is chunked into dot products of TPB-long vectors.
    tmp = 0.
    for i in range(bpg):
        # Preload data into shared memory
        sA[tx, ty] = A[x, ty + i * TPB]
        sB[tx, ty] = B[tx + i * TPB, y]

        # Wait until all threads finish preloading
        cuda.syncthreads()

        # Computes partial product on the shared memory
        for j in range(TPB):
            tmp += sA[tx, j] * sB[j, ty]

        # Wait until all threads finish computing
        cuda.syncthreads()

    C[x, y] = tmp

In [2]:
import numpy as np


x = np.arange(16).reshape(4, 4)
y = np.arange(16).reshape(4, 4)

In [3]:
fast_matmul(x, y)

ValueError: 
Kernel launch configuration was not specified. Use the syntax:

kernel_function[blockspergrid, threadsperblock](arg0, arg1, ..., argn)

See https://numba.pydata.org/numba-doc/latest/cuda/kernels.html#kernel-invocation for help.

